In [3]:
import torch
from botorch.models import SingleTaskGP
from botorch.models.transforms import Standardize
from botorch.acquisition import qExpectedImprovement
from botorch.sampling import SobolQMCNormalSampler, IIDNormalSampler
from botorch.optim import optimize_acqf
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood

# Step 1: Generate sample training data
train_X = torch.rand(10, 1) * 10  # 10 data points in 1D space
train_Y = torch.sin(train_X) + 0.2 * torch.randn_like(train_X)

# Step 2: Define and fit a GP model
gp_model = SingleTaskGP(train_X, train_Y, outcome_transform=Standardize(m=1))
mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)
fit_gpytorch_model(mll)

# Step 3: Define the candidate points for one-step look-ahead
X = torch.rand(20, 1) * 10  # 20 candidate points to fantasize on

# Step 4: Create fantasy scenarios with `fantasize`
# Configure the sampler for generating 1 fantasy samples
sampler = SobolQMCNormalSampler(sample_shape=1)  # num_fantasies = 10

# Generate fantasy model using the candidate points
fantasy_model = gp_model.fantasize(X[:, None, :], sampler=sampler, observation_noise=False)  # fantasy_model.train_targets will have shape (20, 1, 1)

# Step 5: Find the best target (max value) across fantasies for one-step EI
best_f = fantasy_model.train_targets.max(dim=-1)[0]  # shape: (20,)

# Step 6: Define the one-step Expected Improvement (EI) under the fantasized model
# Here we use `qExpectedImprovement` for the fantasized model
one_step_ei = qExpectedImprovement(model=fantasy_model, best_f=best_f)

# Step 7: Optimize the acquisition function to select the next best candidate
bounds = torch.tensor([[0.0], [10.0]])  # Define bounds for optimization
candidate, acq_value = optimize_acqf(
    acq_function=one_step_ei,
    bounds=bounds,
    q=1,  # Choosing 1 point for the next evaluation
    num_restarts=5,
    raw_samples=20,
)

print("Next candidate point for evaluation:", candidate)
print("Acquisition value:", acq_value)


/home/zongchen/miniconda3/envs/look_ahead_bo/lib/python3.10/site-packages/botorch/models/gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)
/home/zongchen/miniconda3/envs/look_ahead_bo/lib/python3.10/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/home/zongchen/miniconda3/envs/look_ahead_bo/lib/python3.10/site-packages/botorch/models/gpytorch.py:106: BotorchTensorDimensionWarning: Non-strict enforcement of botorch tensor conventions. The following error would have been raised with strict enforcement: Expected X and Y to have the same number of 

RuntimeError: Shape mismatch: objects cannot be broadcast to a single shape

In [4]:
fantasy_model.train_targets.shape

torch.Size([1, 20, 11])